In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


In [ ]:
PROJECT_ID = "

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)


In [ ]:
query_validate = """
WITH base_cohort AS (
  SELECT DISTINCT
  ie.subject_id,
  ie.hadm_id,
  ie.stay_id,
  icu.los,
  icu.first_careunit,
  adm.admission_type,
  p.dod,
  ie.caregiver_id,
  h.valuenum AS pre_transfusion_hgb,
  CASE
  WHEN h.valuenum >= 8 THEN 'Early'
  WHEN h.valuenum < 8 THEN 'Late'
  END AS transfusion_group
  FROM physionet-data.mimiciv_3_1_icu.inputevents ie
  INNER JOIN physionet-data.mimiciv_3_1_icu.icustays icu
  ON ie.stay_id = icu.stay_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.admissions adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.patients p
  ON ie.subject_id = p.subject_id
  LEFT JOIN physionet-data.mimiciv_3_1_hosp.labevents h
  ON ie.subject_id = h.subject_id
  AND ie.hadm_id = h.hadm_id
  AND h.itemid IN (50811, 51222, 51640)
  AND h.charttime < ie.starttime
  AND h.charttime >= TIMESTAMP_SUB(ie.starttime, INTERVAL 24 HOUR)
  AND h.valuenum BETWEEN 2 AND 25
  WHERE ie.itemid in (225168,226368, 227070, 221013)
  AND ie.hadm_id IS NOT NULL
  AND h.valuenum IS NOT NULL
  AND ie.statusdescription != 'Rewritten' )
  SELECT transfusion_group,
  COUNT(DISTINCT subject_id) AS n_patients,
  ROUND(AVG(pre_transfusion_hgb), 2) AS avg_hgb,
  ROUND(AVG(los), 1) AS avg_los_days,
  ROUND(COUNT(DISTINCT CASE WHEN dod IS NOT NULL THEN subject_id END) * 100.0
  / COUNT(DISTINCT subject_id), 1) AS mortality_pct
  FROM base_cohort
  GROUP BY transfusion_group
  ORDER BY transfusion_group
"""


In [ ]:
import pandas as pd

df = client.query(query_validate).to_dataframe()
df.head()

,transfusion_group,n_patients,avg_hgb,avg_los_days,mortality_pct
0,Early,12349,9.64,9.3,41.3
1,Late,12113,7.01,11.0,47.8


In [ ]:
df.to_csv("transfusion_query1_results.csv", index=False)

In [ ]:
query_validate1 = """
WITH blood_products AS (
    SELECT
        itemid,
        label
    FROM `physionet-data.mimiciv_3_1_icu.d_items`
    WHERE itemid in (225168, 226368, 227070, 221013)
),

transfusions AS (
    SELECT DISTINCT
        ie.subject_id,
        ie.hadm_id,
        ie.stay_id,
        ie.starttime AS transfusion_time,
        ie.itemid,
        di.label AS product_name,
        ie.amount,
        ie.amountuom
    FROM `physionet-data.mimiciv_3_1_icu.inputevents` ie
    INNER JOIN blood_products bp
        ON ie.itemid = bp.itemid
    INNER JOIN `physionet-data.mimiciv_3_1_icu.d_items` di
        ON ie.itemid = di.itemid
    WHERE ie.amount > 0
    AND ie.hadm_id IS NOT NULL

),

hemoglobin_labs AS (
    SELECT
        le.subject_id,
        le.hadm_id,
        le.charttime,
        le.valuenum AS hb_value,
        le.valueuom
    FROM `physionet-data.mimiciv_3_1_hosp.labevents` le
    INNER JOIN `physionet-data.mimiciv_3_1_hosp.d_labitems` dli
        ON le.itemid = dli.itemid
    WHERE  le.valuenum IS NOT NULL
        AND dli.itemid in  (51640,50811,51222)
        AND le.valuenum > 2
        AND le.valuenum < 25
),

transfusion_with_hb AS (
    SELECT
        t.subject_id,
        t.hadm_id,
        t.stay_id,
        t.transfusion_time,
        t.product_name,
        t.amount,
        t.amountuom,
        h.hb_value,
        h.valueuom AS hb_uom,
        h.charttime AS hb_time,
        TIMESTAMP_DIFF(t.transfusion_time, h.charttime, HOUR) AS hours_before_transfusion,
        ROW_NUMBER() OVER (
            PARTITION BY t.subject_id, t.hadm_id, t.transfusion_time
            ORDER BY ABS(TIMESTAMP_DIFF(t.transfusion_time, h.charttime, MINUTE))
        ) AS rn
    FROM transfusions t
    LEFT JOIN hemoglobin_labs h
        ON t.subject_id = h.subject_id
        AND t.hadm_id = h.hadm_id
        AND t.hadm_id IS NOT NULL
        AND h.charttime <= t.transfusion_time
        AND TIMESTAMP_DIFF(t.transfusion_time, h.charttime, HOUR) <= 24
)
SELECT
    CASE
        WHEN hb_value < 8 THEN '<8 g/dL'
        WHEN hb_value >= 8 AND hb_value < 9 THEN '8-9 g/dL'
        WHEN hb_value >= 9 AND hb_value < 10 THEN '9-10 g/dL'
        WHEN hb_value >= 10 AND hb_value < 11 THEN '10-11 g/dL'
        WHEN hb_value >= 11 AND hb_value < 12 THEN '11-12 g/dL'
        WHEN hb_value >= 12 AND hb_value < 13 THEN '12-13 g/dL'
        ELSE '>=13 Possible Outliers'
    END AS hb_range,
    COUNT(DISTINCT subject_id) AS num_patients,
    COUNT(DISTINCT hadm_id) AS num_admissions,
    COUNT(*) AS num_transfusions,
    ROUND(AVG(hb_value), 2) AS avg_hb_in_range
FROM transfusion_with_hb
WHERE rn = 1
GROUP BY hb_range
ORDER BY
    CASE
        WHEN hb_range = '<8 g/dL' THEN 1
        WHEN hb_range = '8-9 g/dL' THEN 2
        WHEN hb_range = '9-10 g/dL' THEN 3
        WHEN hb_range = '10-11 g/dL' THEN 4
        WHEN hb_range = '11-12 g/dL' THEN 5
        WHEN hb_range = '12-13 g/dL' THEN 6
        WHEN hb_range = '≥13 g/dL' THEN 7
        ELSE 8
    END
"""

In [ ]:
df1 = client.query(query_validate1).to_dataframe()
df1.head(10)

,hb_range,num_patients,num_admissions,num_transfusions,avg_hb_in_range
0,<8 g/dL,10323,11805,34189,6.90
1,8-9 g/dL,5815,6177,13770,8.43
2,9-10 g/dL,4179,4341,8972,9.40
3,10-11 g/dL,2313,2363,4288,10.38
4,11-12 g/dL,1090,1097,1723,11.38
5,12-13 g/dL,431,432,657,12.35
6,>=13 Possible Outliers,2121,2260,3700,13.95


In [ ]:
df1.to_csv("transfusion_query2_results.csv", index=False)

In [ ]:
query_validate2 = """
WITH base_cohort AS (
  SELECT DISTINCT
  ie.subject_id,
  ie.hadm_id,
  ie.stay_id,
  icu.los,
  icu.first_careunit,
  adm.admission_type,
  p.dod,
  p.anchor_year,
  p.gender,
  ie.caregiver_id,
  h.valuenum AS pre_transfusion_hgb,
  CASE
  WHEN h.valuenum >= 8 THEN 'Early'
  WHEN h.valuenum < 8 THEN 'Late'
  END AS transfusion_group,
  CASE
  WHEN p.gender = 'F' THEN 'Female'
  WHEN p.gender = 'M' THEN 'Male'
  END AS gender_group,
  CASE
  WHEN p.anchor_age < 40 THEN '<40 years'
  WHEN p.anchor_age >= 40 AND p.anchor_age < 60  THEN '40-59 years'
  WHEN p.anchor_age >= 60 AND p.anchor_age < 75  THEN '60-74 years'
  WHEN p.anchor_age >= 75   THEN '>= 75 years'
  END AS age_group
  FROM physionet-data.mimiciv_3_1_icu.inputevents ie
  INNER JOIN physionet-data.mimiciv_3_1_icu.icustays icu
  ON ie.stay_id = icu.stay_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.admissions adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.patients p
  ON ie.subject_id = p.subject_id
  LEFT JOIN physionet-data.mimiciv_3_1_hosp.labevents h
  ON ie.subject_id = h.subject_id
  AND ie.hadm_id = h.hadm_id
  AND h.itemid IN (50811, 51222, 51640)
  AND h.charttime < ie.starttime
  AND h.charttime >= TIMESTAMP_SUB(ie.starttime, INTERVAL 24 HOUR)
  AND h.valuenum BETWEEN 2 AND 25
  WHERE ie.itemid in (225168,226368, 227070, 221013)
  AND ie.hadm_id IS NOT NULL
  AND h.valuenum IS NOT NULL
  AND ie.statusdescription != 'Rewritten' )
  SELECT transfusion_group,gender_group,age_group,
  COUNT(DISTINCT subject_id) AS n_patients,
  COUNT(DISTINCT hadm_id) AS n_admissions,
  ROUND(AVG(pre_transfusion_hgb), 2) AS avg_hgb,
  ROUND(AVG(los), 1) AS avg_los_days,
  ROUND(COUNT(DISTINCT CASE WHEN dod IS NOT NULL THEN subject_id END) * 100.0
  / COUNT(DISTINCT subject_id), 1) AS mortality_pct
  FROM base_cohort
  GROUP BY transfusion_group, gender_group, age_group
  ORDER BY transfusion_group, gender_group, age_group
"""


In [ ]:
df2 = client.query(query_validate2).to_dataframe()
df2.head(20)

,transfusion_group,gender_group,age_group,n_patients,n_admissions,avg_hgb,avg_los_days,mortality_pct
0,Early,Female,40-59 years,1219,1355,9.50,8.6,38.3
1,Early,Female,60-74 years,1915,2085,9.55,9.9,37.8
2,Early,Female,<40 years,416,446,9.63,9.6,22.6
3,Early,Female,>= 75 years,1914,2015,9.61,6.9,46.5
4,Early,Male,40-59 years,1802,2014,9.70,10.5,42.1
5,Early,Male,60-74 years,2652,2932,9.71,8.7,41.6
6,Early,Male,<40 years,481,521,9.71,14.6,25.4
7,Early,Male,>= 75 years,1950,2093,9.59,7.9,48.4
8,Late,Female,40-59 years,1348,1608,6.95,11.4,43.8
9,Late,Female,60-74 years,2037,2301,6.97,10.9,46.9


In [ ]:
query_validate3 = """
WITH base_cohort AS (
  SELECT DISTINCT
  ie.subject_id,
  ie.hadm_id,
  ie.stay_id,
  icu.los,
  icu.first_careunit,
  adm.admission_type,
  p.dod,
  ie.caregiver_id,
  h.valuenum AS pre_transfusion_hgb,
  CASE
  WHEN h.valuenum >= 8 THEN 'Early'
  WHEN h.valuenum < 8 THEN 'Late'
  END AS transfusion_group,
  CASE
  WHEN p.gender = 'F' THEN 'Female'
  WHEN p.gender = 'M' THEN 'Male'
  END AS gender_group,
  CASE
  WHEN p.anchor_age < 40 THEN '<40 years'
  WHEN p.anchor_age >= 40 AND p.anchor_age < 60  THEN '40-59 years'
  WHEN p.anchor_age >= 60 AND p.anchor_age < 75  THEN '60-74 years'
  WHEN p.anchor_age >= 75   THEN '>= 75 years'
  END AS age_group,
  CASE
    WHEN adm.race LIKE '%WHITE%' THEN 'White'
    WHEN adm.race LIKE '%BLACK%' OR adm.race LIKE '%AFRICAN%' THEN 'Black/African American'
    WHEN adm.race LIKE '%ASIAN%' THEN 'Asian'
    WHEN adm.race LIKE '%HISPANIC%' OR adm.race LIKE '%LATINO%' THEN 'Hispanic/Latino'
    ELSE 'Other/Unknown'
  END AS race_group
  FROM physionet-data.mimiciv_3_1_icu.inputevents ie
  INNER JOIN physionet-data.mimiciv_3_1_icu.icustays icu
  ON ie.stay_id = icu.stay_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.admissions adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.patients p
  ON ie.subject_id = p.subject_id
  LEFT JOIN physionet-data.mimiciv_3_1_hosp.labevents h
  ON ie.subject_id = h.subject_id
  AND ie.hadm_id = h.hadm_id
  AND h.itemid IN (50811, 51222, 51640)
  AND h.charttime < ie.starttime
  AND h.charttime >= TIMESTAMP_SUB(ie.starttime, INTERVAL 24 HOUR)
  AND h.valuenum BETWEEN 2 AND 25
  WHERE ie.itemid in (225168,226368, 227070, 221013)
  AND ie.hadm_id IS NOT NULL
  AND h.valuenum IS NOT NULL
  AND ie.statusdescription != 'Rewritten' )
  SELECT transfusion_group,gender_group,age_group,race_group,
  COUNT(DISTINCT subject_id) AS n_patients,
  ROUND(AVG(pre_transfusion_hgb), 2) AS avg_hgb,
  ROUND(AVG(los), 1) AS avg_los_days,
  ROUND(COUNT(DISTINCT CASE WHEN dod IS NOT NULL THEN subject_id END) * 100.0
  / COUNT(DISTINCT subject_id), 1) AS mortality_pct
  FROM base_cohort
  GROUP BY transfusion_group,gender_group,age_group,race_group
  ORDER BY transfusion_group,gender_group,age_group,race_group
  """

In [ ]:
df3 = client.query(query_validate3).to_dataframe()
df3.head(1000)

,transfusion_group,gender_group,age_group,race_group,n_patients,avg_hgb,avg_los_days,mortality_pct
0,Early,Female,40-59 years,Asian,55,9.62,4.7,40.0
1,Early,Female,40-59 years,Black/African American,163,9.37,7.9,45.4
2,Early,Female,40-59 years,Hispanic/Latino,64,9.40,8.2,31.3
3,Early,Female,40-59 years,Other/Unknown,180,9.32,9.0,43.9
4,Early,Female,40-59 years,White,766,9.56,8.9,36.6
...,...,...,...,...,...,...,...,...
75,Late,Male,>= 75 years,Asian,75,6.97,9.5,62.7
76,Late,Male,>= 75 years,Black/African American,94,6.88,13.8,77.7
77,Late,Male,>= 75 years,Hispanic/Latino,40,6.98,8.4,57.5
78,Late,Male,>= 75 years,Other/Unknown,318,7.14,8.7,49.7


In [ ]:
query_validate4 = """
WITH base_cohort AS (
  SELECT DISTINCT
  ie.subject_id,
  ie.hadm_id,
  ie.stay_id,
  icu.los,
  icu.first_careunit,
  adm.admission_type,
  adm.race,
  p.dod,
  p.anchor_year,
  p.gender,
  ie.caregiver_id,
  h.valuenum AS pre_transfusion_hgb,
  h.charttime,
  ie.starttime,
  CASE
  WHEN h.valuenum >= 8 THEN 'Early'
  WHEN h.valuenum < 8 THEN 'Late'
  END AS transfusion_group,
  CASE
  WHEN p.gender = 'F' THEN 'Female'
  WHEN p.gender = 'M' THEN 'Male'
  END AS gender_group,
  CASE
  WHEN p.anchor_age < 40 THEN '<40 years'
  WHEN p.anchor_age >= 40 AND p.anchor_age < 60  THEN '40-59 years'
  WHEN p.anchor_age >= 60 AND p.anchor_age < 75  THEN '60-74 years'
  WHEN p.anchor_age >= 75   THEN '>= 75 years'
  END AS age_group,
  CASE
    WHEN adm.race LIKE '%WHITE%' THEN 'White'
    WHEN adm.race LIKE '%BLACK%' OR adm.race LIKE '%AFRICAN%' THEN 'Black/African American'
    WHEN adm.race LIKE '%ASIAN%' THEN 'Asian'
    WHEN adm.race LIKE '%HISPANIC%' OR adm.race LIKE '%LATINO%' THEN 'Hispanic/Latino'
    ELSE 'Other/Unknown'
  END AS race_group
  FROM physionet-data.mimiciv_3_1_icu.inputevents ie
  INNER JOIN physionet-data.mimiciv_3_1_icu.icustays icu
  ON ie.stay_id = icu.stay_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.admissions adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.patients p
  ON ie.subject_id = p.subject_id
  LEFT JOIN physionet-data.mimiciv_3_1_hosp.labevents h
  ON ie.subject_id = h.subject_id
  AND ie.hadm_id = h.hadm_id
  AND h.itemid IN (50811, 51222, 51640)
  AND h.charttime < ie.starttime
  AND h.charttime >= TIMESTAMP_SUB(ie.starttime, INTERVAL 24 HOUR)
  AND h.valuenum BETWEEN 2 AND 25
  WHERE ie.itemid in (225168,226368, 227070, 221013)
  AND ie.hadm_id IS NOT NULL
  AND h.valuenum IS NOT NULL
  AND ie.statusdescription != 'Rewritten'
),
-- Create a unique record per patient/admission/transfusion event
unique_records AS (
  SELECT
    subject_id,
    hadm_id,
    stay_id,
    los,
    dod,
    gender_group,
    age_group,
    race_group,
    transfusion_group,
    pre_transfusion_hgb,
    ROW_NUMBER() OVER (PARTITION BY subject_id, hadm_id, transfusion_group ORDER BY charttime) as rn
  FROM base_cohort
)
SELECT
  transfusion_group,
  gender_group,
  age_group,
  race_group,
  COUNT(DISTINCT subject_id) AS n_patients,
  COUNT(DISTINCT hadm_id) AS n_admissions,
  ROUND(AVG(pre_transfusion_hgb), 2) AS avg_hgb,
  ROUND(AVG(los), 1) AS avg_los_days,
  ROUND(COUNT(DISTINCT CASE WHEN dod IS NOT NULL THEN subject_id END) * 100.0
  / COUNT(DISTINCT subject_id), 1) AS mortality_pct
FROM unique_records
WHERE rn = 1
GROUP BY transfusion_group, gender_group, age_group, race_group
ORDER BY transfusion_group, gender_group, age_group, race_group
"""

In [ ]:

df4 = client.query(query_validate4).to_dataframe()
df4.head(1000)


,transfusion_group,gender_group,age_group,race_group,n_patients,n_admissions,avg_hgb,avg_los_days,mortality_pct
0,Early,Female,40-59 years,Asian,55,63,9.81,4.8,40.0
1,Early,Female,40-59 years,Black/African American,163,190,9.31,6.2,45.4
2,Early,Female,40-59 years,Hispanic/Latino,64,67,9.66,7.3,31.3
3,Early,Female,40-59 years,Other/Unknown,180,189,9.48,7.6,43.9
4,Early,Female,40-59 years,White,766,846,9.69,6.2,36.6
...,...,...,...,...,...,...,...,...,...
75,Late,Male,>= 75 years,Asian,75,84,7.06,6.7,62.7
76,Late,Male,>= 75 years,Black/African American,94,104,6.99,6.8,77.7
77,Late,Male,>= 75 years,Hispanic/Latino,40,46,7.07,6.7,57.5
78,Late,Male,>= 75 years,Other/Unknown,318,326,7.21,6.7,49.7


In [ ]:
query_validate5 = """
WITH base_cohort AS (
  SELECT DISTINCT
  ie.subject_id,
  ie.hadm_id,
  ie.stay_id,
  icu.los,
  icu.first_careunit,
  adm.admission_type,
  adm.race,
  p.dod,
  p.anchor_year,
  p.gender,
  ie.caregiver_id,
  h.valuenum AS pre_transfusion_hgb,
  h.charttime,
  ie.starttime,
  CASE
  WHEN h.valuenum >= 8 THEN 'Early'
  WHEN h.valuenum < 8 THEN 'Late'
  END AS transfusion_group,
  CASE
  WHEN p.gender = 'F' THEN 'Female'
  WHEN p.gender = 'M' THEN 'Male'
  END AS gender_group,
  CASE
  WHEN p.anchor_age < 40 THEN '<40 years'
  WHEN p.anchor_age >= 40 AND p.anchor_age < 60  THEN '40-59 years'
  WHEN p.anchor_age >= 60 AND p.anchor_age < 75  THEN '60-74 years'
  WHEN p.anchor_age >= 75   THEN '>= 75 years'
  END AS age_group,
  CASE
    WHEN adm.race LIKE '%WHITE%' THEN 'White'
    WHEN adm.race LIKE '%BLACK%' OR adm.race LIKE '%AFRICAN%' THEN 'Black/African American'
    WHEN adm.race LIKE '%ASIAN%' THEN 'Asian'
    WHEN adm.race LIKE '%HISPANIC%' OR adm.race LIKE '%LATINO%' THEN 'Hispanic/Latino'
    ELSE 'Other/Unknown'
  END AS race_group
 FROM physionet-data.mimiciv_3_1_icu.inputevents ie
  INNER JOIN physionet-data.mimiciv_3_1_icu.icustays icu
  ON ie.stay_id = icu.stay_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.admissions adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.patients p
  ON ie.subject_id = p.subject_id
  LEFT JOIN physionet-data.mimiciv_3_1_hosp.labevents h
  ON ie.subject_id = h.subject_id
  AND ie.hadm_id = h.hadm_id
  AND h.itemid IN (50811, 51222, 51640)
  AND h.charttime < ie.starttime
  AND h.charttime >= TIMESTAMP_SUB(ie.starttime, INTERVAL 24 HOUR)
  AND h.valuenum BETWEEN 2 AND 25
  WHERE ie.itemid in (225168,226368, 227070, 221013)
  AND ie.hadm_id IS NOT NULL
  AND h.valuenum IS NOT NULL
  AND ie.statusdescription != 'Rewritten'),
-- Create a unique record per patient/admission/transfusion event
unique_records AS (
  SELECT
    subject_id,
    hadm_id,
    stay_id,
    los,
    dod,
    gender_group,
    age_group,
    race_group,
    transfusion_group,
    pre_transfusion_hgb,
    ROW_NUMBER() OVER (PARTITION BY subject_id, hadm_id, transfusion_group ORDER BY charttime) as rn
  FROM base_cohort
)
SELECT
  race_group,
  COUNT(DISTINCT subject_id) AS n_patients,
  COUNT(DISTINCT hadm_id) AS n_admissions,
  ROUND(AVG(pre_transfusion_hgb), 2) AS avg_hgb,
  ROUND(AVG(los), 1) AS avg_los_days,
  ROUND(COUNT(DISTINCT CASE WHEN dod IS NOT NULL THEN subject_id END) * 100.0
  / COUNT(DISTINCT subject_id), 1) AS mortality_pct
FROM unique_records
WHERE rn = 1
GROUP BY race_group
ORDER BY race_group
"""

In [ ]:

df5 = client.query(query_validate5).to_dataframe()
df5.head(1000)


,race_group,n_patients,n_admissions,avg_hgb,avg_los_days,mortality_pct
0,Asian,535,628,8.43,6.3,45.6
1,Black/African American,1495,1863,8.02,6.7,51.2
2,Hispanic/Latino,589,708,8.50,6.5,39.4
3,Other/Unknown,2783,2912,8.59,7.2,41.6
4,White,10734,12371,8.54,6.2,44.7


In [ ]:
query_validate6 = """
WITH base_cohort AS (
  SELECT DISTINCT
  ie.subject_id,
  ie.hadm_id,
  ie.stay_id,
  icu.los,
  icu.first_careunit,
  adm.admission_type,
  p.dod,
  p.anchor_year,
  p.gender,
  ie.caregiver_id,
  h.valuenum AS pre_transfusion_hgb,
  CASE
  WHEN h.valuenum >= 8 THEN 'Early'
  WHEN h.valuenum < 8 THEN 'Late'
  END AS transfusion_group,
  CASE
  WHEN p.gender = 'F' THEN 'Female'
  WHEN p.gender = 'M' THEN 'Male'
  ELSE "Others"
  END AS gender_group,
  CASE
  WHEN p.anchor_age < 40 THEN '<40 years'
  WHEN p.anchor_age >= 40 AND p.anchor_age < 60  THEN '40-59 years'
  WHEN p.anchor_age >= 60 AND p.anchor_age < 75  THEN '60-74 years'
  WHEN p.anchor_age >= 75   THEN '>= 75 years'
  END AS age_group,
  CASE
    WHEN adm.race LIKE '%WHITE%' THEN 'White'
    WHEN adm.race LIKE '%BLACK%' OR adm.race LIKE '%AFRICAN%' THEN 'Black/African American'
    WHEN adm.race LIKE '%ASIAN%' THEN 'Asian'
    WHEN adm.race LIKE '%HISPANIC%' OR adm.race LIKE '%LATINO%' THEN 'Hispanic/Latino'
    ELSE 'Other/Unknown'
  END AS race_group
 FROM physionet-data.mimiciv_3_1_icu.inputevents ie
  INNER JOIN physionet-data.mimiciv_3_1_icu.icustays icu
  ON ie.stay_id = icu.stay_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.admissions adm
  ON ie.hadm_id = adm.hadm_id
  INNER JOIN physionet-data.mimiciv_3_1_hosp.patients p
  ON ie.subject_id = p.subject_id
  LEFT JOIN physionet-data.mimiciv_3_1_hosp.labevents h
  ON ie.subject_id = h.subject_id
  AND ie.hadm_id = h.hadm_id
  AND h.itemid IN (50811, 51222, 51640)
  AND h.charttime < ie.starttime
  AND h.charttime >= TIMESTAMP_SUB(ie.starttime, INTERVAL 24 HOUR)
  AND h.valuenum BETWEEN 2 AND 25
  WHERE ie.itemid in (225168,226368, 227070, 221013)
  AND ie.hadm_id IS NOT NULL
  AND h.valuenum IS NOT NULL
  AND ie.statusdescription != 'Rewritten' )
  SELECT
  COUNT(DISTINCT subject_id) AS n_patients,
  COUNT(DISTINCT hadm_id) AS n_admissions,
  ROUND(AVG(pre_transfusion_hgb), 2) AS avg_hgb,
  ROUND(AVG(los), 1) AS avg_los_days,
  ROUND(COUNT(DISTINCT CASE WHEN dod IS NOT NULL THEN subject_id END) * 100.0
  / COUNT(DISTINCT subject_id), 1) AS mortality_pct
  FROM base_cohort
  """

In [ ]:

df6 = client.query(query_validate6).to_dataframe()
df6.head(1000)


,n_patients,n_admissions,avg_hgb,avg_los_days,mortality_pct
0,16059,18482,8.54,10.0,44.6
